# 1. 
Comme $M$ est une matrice symmétrique définie positive, ses valeurs propres seront toutes réelles et positives et égales à ses valeurs singulières. Les valeurs propres de $M$ respecteront $Mv= cv$ pour une valeur propre $c$ réelle positive. Si on cherche maintenant les valeurs propres de $M + \lambda² I$, on peut voir que
$$
\begin{align}
 (M + \lambda² I)v &= dv\\
  Mv + \lambda²v &= dv\\
  Mv &= dv - \lambda²v\\
  cv &= dv - \lambda²v\\
  d &= c + \lambda²
\end{align}
$$

Donc les valeurs propres (et singulières) de $M + \lambda² I$ sont celles de $M$ mais décalé de $\lambda²$. De cela, si le nombre de conditionnement de $M$ est 
$$ \kappa(M) = \sigma_\text{max}(M) / \sigma_\text{min}(M) $$

alors 
$$ \kappa(M + \lambda²) = (\sigma_\text{max}(M) + \lambda²) / (\sigma_\text{min}(M)+ \lambda²)$$

Or comme $\lambda² > 0$, cela implique que $ \kappa(M) > \kappa(M + \lambda²)$.

# 2. 

# 3. 

# 4. 

# 5. 

# 6.

# 7. 

In [1]:
using LinearAlgebra
using SparseArrays
using Printf
using Test

In [2]:
function test_least_square(f::Function, minimum_norm::Bool)
    A_1 = [1. 2. 3.; 4. 5. 6.; 7. 8. 9.; 5. 7. 9.]
    b_1 = ones(Float64, 4)
    
    x = f(sparse(A_1), b_1)
    x_baseline = A_1 \ b_1

    r = A_1 * x - b_1
    r_baseline = A_1 * x_baseline - b_1
    @test norm(r - r_baseline) ≈ 0.0 atol=1e-6   

    if minimum_norm
        @test norm(x - x_baseline) ≈ 0.0 atol=1e-6
    end
end

test_least_square (generic function with 1 method)

In [51]:
function golub_riley(A::SparseMatrixCSC{Float64}, b::Vector{Float64}, λ::Float64=0.1, ϵ::Float64=1e-4)
    m, n = size(A)
    x_k = zeros(Float64, n)
    b_k = [b; zeros(n)]
    A_augmented = sparse_vcat(A, sparse(λ * I, n, n))
    QR_A_augmented = qr(A_augmented)

    Δx_k = Vector{Float64}(undef, n)
    while true
        # We use \ because ldiv!(Y, A, b) does not seem to be implemented with sparse QR factorization
        Δx_k = QR_A_augmented \ b_k
        x_k += Δx_k
        b_k[1:m] = b - A * x_k
        b_k[m+1:m+n] .= 0.0
        if (norm(Δx_k) / norm(x_k)) ≤ ϵ
            break
        end
    end
    
    return x_k
end


golub_riley (generic function with 3 methods)

In [52]:
test_least_square(golub_riley, false)

# 8. 

In [5]:
using HarwellRutherfordBoeing

In [6]:
function get_problem_from_animal(path_to_animal_folder::String, problem_name::String)
    A = HarwellBoeingMatrix(joinpath(path_to_animal_folder, "hb", problem_name * ".hb"))
    return A.matrix, vec(A.rhs)
end

get_problem_from_animal (generic function with 1 method)

In [7]:
function get_solution_from_animal(path_to_animal_folder::String, problem_name::String)
    path = joinpath(path_to_animal_folder, "mls", "txt", problem_name * "_scaled_mls.txt")
    x = map(x -> parse(Float64, x), readlines(path))
    return x
end

get_solution_from_animal (generic function with 1 method)

# 9. 

In [8]:
function scale_columns_to_unit_norm!(A::SparseMatrixCSC{Float64})
    for j = 1:A.n
        col_norm = norm(A[:, j])
        A[:, j] ./= col_norm
    end
end

scale_columns_to_unit_norm! (generic function with 1 method)

In [9]:
# Tests
for n = 10:30
    A = sprandn(Float64, n, n, 0.5)
    scale_columns_to_unit_norm!(A)
    for j = 1:n
        @test norm(A[:, j]) ≈ 1.0
    end
end

# 10. 

In [10]:
using BenchmarkTools
using PrettyTables

In [53]:
# very2 has no solution in the animal/mls folder so we ignore it.
animal_problem = ["small", "small2", "medium", "medium2", "large", "large2", "very"]
for problem in animal_problem

    # Replace "animal" by the path of the animal problem folder on your machine.
    A, b = get_problem_from_animal("animal", problem)
    scale_columns_to_unit_norm!(A)

    x = golub_riley(A, b)
    r = b - A * x
    median_time = median(@benchmark(golub_riley($A, $b)))
    median_time_baseline = median(@benchmark($A \ $b))
    @show median_time
    @show median_time_baseline

    x_reference = get_solution_from_animal("animal", problem)
    r_reference = b - A * x_reference

    x_relative_error = norm(x - x_reference) / norm(x_reference)
    println(x_relative_error)

    r_relative_error = norm(r - r_reference) / norm(r_reference)
    println(r_relative_error)
    println("----------")
end

median_time = TrialEstimate(9.318 ms)
median_time_baseline = TrialEstimate(4.230 ms)
0.0003887972070944706
0.00027439390172447455
----------
median_time = TrialEstimate(1.282 s)
median_time_baseline = TrialEstimate(17.512 ms)
0.039656585137865394
0.0031963284299092133
----------
median_time = TrialEstimate(128.903 ms)
median_time_baseline = TrialEstimate(61.751 ms)
0.0002946631571742394
0.0002541106867997859
----------


LoadError: InterruptException:

# 11. 

Si on part du système
$$
\begin{pmatrix}
I & A \\
A^T & - \lambda^{2}I
\end{pmatrix}
\begin{pmatrix}
\Delta r_k \\ \Delta x_k
\end{pmatrix}
=
\begin{pmatrix}
b_k \\ 0
\end{pmatrix}
$$

on obtient les égalités $\Delta r_k + A \Delta x_k = b_k$ et $A^T \Delta r_k - \lambda^{2} \Delta x_k =0$. De là, on sait que $\Delta r_k = b_k - A \Delta x_k$. En substituant $\Delta r_k$ dans la deuxième équation par $b_k - A \Delta x_k$, on obtient
$$
\begin{align*}
A^T( b_k - A \Delta x_k) - \lambda^{2} x_k &= 0\\
A^T A \Delta x_k + \lambda^{2} \Delta x_k &= A^T b_k\\
(A^T A + \lambda^{2}I)  \Delta x_k = A^T b_k
\end{align*}
$$

On peut faire la même chose en partant de $(A^T A + \lambda^{2}I)  \Delta x_k = A^T b_k$, il faut seulement faire les étapes en sens inverses et ajouter une contrainte pour définir la variable $\Delta r_k$.

# 12. 

On peut se servir du fait que deux matrices congruentes ont la même intertie. Soit 
$$ K = 
\begin{pmatrix}
I & A \\
A^T & - \lambda^{2}I
\end{pmatrix}
$$

On cherche alors une matrice $D$ congruente à la matrice bloc $K$, ce qui veut dire que $K = M^*DM$ où $M$ est inversible. Regardons
\begin{equation*}
  K = M^* D M =
  \left(
    \begin{array}{cc}
    I & 0 \\
    A^* & I
    \end{array}
  \right)
  \left(
  \begin{array}{cc}
    I & 0 \\
    0 & -A^* A - \lambda^{2} I 
  \end{array}
\right)
\left(
  \begin{array}{cc}
    I & A \\
    0 & I        
  \end{array}
  \right)
\end{equation*}

On calcule aisément que cette égalité tient. On voit également que $M$ et $M^*$ sont inversibles car elles ont un déterminant de $1$ car elles n'ont que des éléments de $1$ sur la diagonale. $K$ est donc congruente à $D$.

Il faut maintenant examiner les valeurs propres de $D$. Par sa structure de bloc uniquement sur la diagonale, les valeurs propres de $D$ seront l'union des valeurs propres de $I$ et de $-A^*A - \lambda^{2}I$. Les valeurs propres de $I$ sont $1$. 
Quant à $-A^*A - \lambda^{2}I$, regardons pour un vecteur $x$
\begin{align*}
x^*(-A^*A - \lambda^{2}I)x &= -x^* A^*A x - \lambda^{2}x^*x\\
&= -||Ax||^{2} - ||x||^{2}
\end{align*}

La norme de $||Ax||$ pourrait être nulle pour un vecteur $x$ non-nulle, mais dans ce cas la norme de $x$ serait positive et donc le membre de droite de l'équation serait négatif. On en conclut que le membre de droite est nulle seulement si $x$ est nul et donc que le bloc en bas à droite de $D$ est défini négatif, et donc que $D$ a seulement des valeurs propres strictement négative ou positive.

Comme $K$ a la même inertie que $D$, elle aura des valeurs propres strictement positives et strictement négatives, ce qui veut dire que la matrice est inversible.


# 13. 

In [18]:
using LDLFactorizations

In [55]:
function golub_riley_2(A::SparseMatrixCSC{Float64}, b::Vector{Float64}, λ::Float64=0.1, ϵ::Float64=1e-4)
    m, n = size(A)
    x_k = zeros(Float64, n)
    b_augmented_k = [b; zeros(n)]

    # We could pass an upper triangular matrix to ldl for more memory efficiency
    K = [I A; adjoint(A) (-(λ^2) * I)]
    LDLT = ldl(K)
    Δsol_k = Vector{Float64}(undef, m + n)
    while true
        ldiv!(Δsol_k, LDLT, b_augmented_k)
        x_k += Δsol_k[m+1:m+n]
        b_augmented_k[1:m] = b - A * x_k
        if (norm(Δsol_k[m+1:m+n]) / norm(x_k)) ≤ ϵ
            break
        end
    end
    
    return x_k
end

golub_riley_2 (generic function with 3 methods)

In [56]:
test_least_square(golub_riley_2, false)

In [57]:
# very2 has no solution in the animal/mls folder so we ignore it.
animal_problem = ["small", "small2", "medium", "medium2", "large", "large2", "very"]
for problem in animal_problem

    # Replace "animal" by the path of the animal problem folder on your machine.
    A, b = get_problem_from_animal("animal", problem)
    scale_columns_to_unit_norm!(A)

    x = golub_riley_2(A, b)
    r = b - A * x
    @btime (golub_riley_2($A, $b))
    @btime ($A \ $b)
    #median_time = median(@benchmark(golub_riley_2($A, $b)))
    #median_time_baseline = median(@benchmark($A \ $b))
    #@show median_time
    #@show median_time_baseline

    x_reference = get_solution_from_animal("animal", problem)
    r_reference = b - A * x_reference

    x_relative_error = norm(x - x_reference) / norm(x_reference)
    println(x_relative_error)

    r_relative_error = norm(r - r_reference) / norm(r_reference)
    println(r_relative_error)
    println("----------")
end

  3.280 ms (366 allocations: 5.20 MiB)
  3.545 ms (196 allocations: 4.89 MiB)
0.0003887972070944183
0.00027439390172444153
----------
  292.401 ms (8036 allocations: 155.32 MiB)
  15.467 ms (205 allocations: 22.25 MiB)
0.03965658513786405
0.0031963284299091036
----------
  37.191 ms (377 allocations: 40.96 MiB)
  54.667 ms (201 allocations: 64.73 MiB)
0.0002946631571742237
0.0002541106867997748
----------
  449.471 ms (437 allocations: 125.98 MiB)
  340.412 ms (206 allocations: 376.63 MiB)
0.31247420147425514
0.00928926298762896
----------
  13.416 ms (416 allocations: 15.38 MiB)
  15.811 ms (201 allocations: 19.46 MiB)
0.0001690724674407964
0.00017165689223808098
----------
  2.428 s (12857 allocations: 746.05 MiB)
  60.575 ms (205 allocations: 82.73 MiB)
0.10893594351497583
0.005452652788888664
----------


LoadError: InterruptException:

# 14. 

# 15. 